In [8]:
# Import python packages
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os

#Lab 2 specific imports
from azure.ai.projects.models import CodeInterpreterTool
from typing import Any
from pathlib import Path


In [9]:
# Load environment variables from .env file
load_dotenv()

project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
model = os.getenv("CHAT_MODEL")

print(project_connection_string)
print(model)

eastus.api.azureml.ms;4c61dc39-311c-43a4-bd17-4b18ec045427;rg-agent;project-demo-v2zb
gpt-4o-mini


In [10]:
# connect to the AI project 
project_client = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, credential=DefaultAzureCredential()
)

In [ ]:
with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent-barchart",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    prompt = "Could you please create a bar chart for the using the following data and \
        provide the file to me? Name the file as health-plan-comparision.png. \
        Here is the data: \
        Provider	    Monthly Premium	Deductible	Out-of-Pocket Limit \
        Northwind	    $300		$1,500		$6,000 \
        Aetna		    $350		$1,000		$5,500 \
        United Health	$250		$2,000		$7,000 \
        Premera		    $200		$2,200		$6,500 \
    "
    
    # Create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Get the last message from the sender
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # Generate an image file for the bar chart
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")

    # Print the file path(s) from the messages
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")
        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)

    # Delete the agent once done
    #project_client.agents.delete_agent(agent.id)
    #print("Deleted agent")

Created agent, agent ID: asst_KNBlkeaG7HBQIJSSFzdumYma
Created thread, thread ID: thread_oYRuZo8gIIHduy5Ym2AsQgE5
Created message, message ID: msg_VCcj5Ikv6wnSDkZ8wiFv5Uk4
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_exjzsBgl3dZnj81MpwhxFG78', 'object': 'thread.message', 'created_at': 1738163180, 'assistant_id': 'asst_KNBlkeaG7HBQIJSSFzdumYma', 'thread_id': 'thread_oYRuZo8gIIHduy5Ym2AsQgE5', 'run_id': 'run_yrwiOMyYcbmzF2Vqcjtr0Fnp', 'role': 'assistant', 'content': [{'type': 'image_file', 'image_file': {'file_id': 'assistant-q3GfmzHydzkUMd8dDBlkm7HF'}}, {'type': 'text', 'text': {'value': 'I have created the bar chart for the health plan comparison. You can download it using the link below:\n\n[Download health-plan-comparision.png](sandbox:/mnt/data/health-plan-comparision.png)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/health-plan-comparision.png', 'start_index': 143, 'end_index': 188, 'file_path': {'file_id': 'a